# Upload multiple CSV files from a Google Cloud Bucket to a Big Query dataset

Package installation

In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

# Authenticate with Google Cloud
auth.authenticate_user()

# Set up BigQuery client
client = bigquery.Client(project='ty-mini-project-41')

# Function to create a new table in BigQuery
def create_table(dataset_id, table_id, schema):
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)


    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    print(f'Table {table.table_id} created successfully.')




In [ ]:
# Function to read a csv file from a Google cloud Bucket and push its content to a table in Big Query
def load_csv_to_table(dataset_id, table_id, csv_path):
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    job_config = bigquery.LoadJobConfig()
    job_config.skip_leading_rows = 1  # Skip the header row

    with open(csv_path, 'rb') as source_file:
        job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

    job.result()  # Wait for the job to complete

    print(f'CSV file {csv_path} loaded into table {table_id} successfully.')

The following functions were included to standarize date formats within the files

In [ ]:
from datetime import datetime
#find object columns

def change_time(x):
    format = '%m/%d/%Y %I:%M:%S %p'
    my_date = datetime.strptime(x, format)
    return my_date


def change_types(x):
  if x.count('int')>0:
    return "NUMERIC"
  elif x.count('date')>0:
    return 'DATETIME'
  elif x.count('float')>0:
    return 'BIGNUMERIC'
  else:
    return 'STRING'

The process need to receive the name of each file that you would like to add, so i included this function to list all files in a google cloud bucket

In [ ]:
from google.cloud import storage

def list_files_in_bucket(bucket_name):
    # Initialize a client
    client = storage.Client()

    for blob in client.list_blobs(bucket_name):
      print(str(blob))


list_files_in_bucket('final_preprocessed_files_when2heat')


<Blob: final_preprocessed_files_when2heat, AT_preprocessed_data.csv, 1714045831210248>
<Blob: final_preprocessed_files_when2heat, BE_preprocessed_data.csv, 1714045923492032>
<Blob: final_preprocessed_files_when2heat, BG_preprocessed_data.csv, 1714045943516977>
<Blob: final_preprocessed_files_when2heat, CH_preprocessed_data.csv, 1714045958313634>
<Blob: final_preprocessed_files_when2heat, CZ_preprocessed_data.csv, 1714045887237287>
<Blob: final_preprocessed_files_when2heat, DE_preprocessed_data.csv, 1714045965651638>
<Blob: final_preprocessed_files_when2heat, DK_preprocessed_data.csv, 1714045942192461>
<Blob: final_preprocessed_files_when2heat, EE_preprocessed_data.csv, 1714045814511687>
<Blob: final_preprocessed_files_when2heat, ES_preprocessed_data.csv, 1714045922241203>
<Blob: final_preprocessed_files_when2heat, FI_preprocessed_data.csv, 1714045907839405>
<Blob: final_preprocessed_files_when2heat, FR_preprocessed_data.csv, 1714045936814214>
<Blob: final_preprocessed_files_when2heat, 

In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

# Authenticate with Google Cloud
auth.authenticate_user()

# Set up BigQuery client
client = bigquery.Client(project='ty-mini-project-41')

# Define the dataset_id
dataset_id = 'final_preprocessed_when2heat'  # Replace with your actual dataset ID

# Function to create a new table in BigQuery
def create_table(dataset_id, table_id, schema):
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    print(f'Table {table.table_id} created successfully.')

# Function to read a csv file from a Google cloud Bucket and push its content to a table in Big Query
def load_csv_to_table(dataset_id, table_id, csv_path):
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)

    job_config = bigquery.LoadJobConfig()
    job_config.skip_leading_rows = 1  # Skip the header row

    with open(csv_path, 'rb') as source_file:
        job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

    job.result()  # Wait for the job to complete

    print(f'CSV file {csv_path} loaded into table {table_id} successfully.')

# Function to fix the CSV DataFrame
from datetime import datetime

def fix_the_csv(df, country_prefix):
    # Combine 'year', 'hour', 'day', and 'month' columns into 'cet_timestamp'
    df['cet_timestamp'] = df.apply(lambda row: datetime(row['year'], row['month'], row['day'], row['hour']), axis=1)


    # Add country prefix to each column except 'cet_timestamp'
    df.columns = [f'{country_prefix}_{col}' if col not in ['hour', 'day', 'month', 'year','cet_timestamp'] else col for col in df.columns]

    return df

# Iterate over the CSV files
file_names = [
    'AT_preprocessed_data.csv',
    'BE_preprocessed_data.csv',
    'BG_preprocessed_data.csv',
    'CH_preprocessed_data.csv',
    'CZ_preprocessed_data.csv',
    'DE_preprocessed_data.csv',
    'DK_preprocessed_data.csv',
    'EE_preprocessed_data.csv',
    'ES_preprocessed_data.csv',
    'FI_preprocessed_data.csv',
    'FR_preprocessed_data.csv',
    'GB_preprocessed_data.csv',
    'GR_preprocessed_data.csv',
    'HR_preprocessed_data.csv',
    'HU_preprocessed_data.csv',
    'IE_preprocessed_data.csv',
    'IT_preprocessed_data.csv',
    'LT_preprocessed_data.csv',
    'LU_preprocessed_data.csv',
    'LV_preprocessed_data.csv',
    'NL_preprocessed_data.csv',
    'NO_preprocessed_data.csv',
    'PL_preprocessed_data.csv',
    'PT_preprocessed_data.csv',
    'RO_preprocessed_data.csv',
    'SE_preprocessed_data.csv',
    'SI_preprocessed_data.csv',
    'SK_preprocessed_data.csv'
]

csv_files = ['gs://final_preprocessed_files_when2heat/'+file for file in file_names]  # Update with your CSV file names and path

for csv_file in csv_files:
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Extract the country prefix from the file name
    country_prefix = csv_file.split('/')[-1][:2]

    # Modify the column names with the country prefix
    df = fix_the_csv(df, country_prefix)

    # Create a schema dictionary
    dtypes_dic = df.dtypes.apply(lambda x: str(x)).to_dict()
    ready_dtypes_dic = {k: change_types(dtypes_dic[k]) for k in dtypes_dic}
    schema = [bigquery.SchemaField(k, ready_dtypes_dic[k]) for k in ready_dtypes_dic]

    # Extract the table name from the CSV file name
    table_id = csv_file.split('/')[-1]
    table_id = table_id.replace(".csv", "")

    # Create a new table in BigQuery
    create_table(dataset_id, table_id, schema)

    # Write the CSV into the newly created table
    df.to_csv(table_id, index=False)  # Save the CSV file with the modified column names

    # Load the CSV into the newly created table
    load_csv_to_table(dataset_id, table_id, table_id)


Table AT_preprocessed_data created successfully.
CSV file AT_preprocessed_data loaded into table AT_preprocessed_data successfully.
Table BE_preprocessed_data created successfully.
CSV file BE_preprocessed_data loaded into table BE_preprocessed_data successfully.
Table BG_preprocessed_data created successfully.
CSV file BG_preprocessed_data loaded into table BG_preprocessed_data successfully.
Table CH_preprocessed_data created successfully.
CSV file CH_preprocessed_data loaded into table CH_preprocessed_data successfully.
Table CZ_preprocessed_data created successfully.
CSV file CZ_preprocessed_data loaded into table CZ_preprocessed_data successfully.
Table DE_preprocessed_data created successfully.
CSV file DE_preprocessed_data loaded into table DE_preprocessed_data successfully.
Table DK_preprocessed_data created successfully.
CSV file DK_preprocessed_data loaded into table DK_preprocessed_data successfully.
Table EE_preprocessed_data created successfully.
CSV file EE_preprocessed_da